<a href="https://colab.research.google.com/github/step-cheng/CIFAR10-pytorch/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch>=2.0.1
!pip install torchvision

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [ ]:

train_data = datasets.CIFAR10(root='./', train=True, download=True, transform=transforms.ToTensor(),
                              target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
# DataLoader wraps an iterable over dataset for automatic batching, sampling, and dataloading
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

test_data = datasets.CIFAR10(root='./', train=False, download=True, transform=transforms.ToTensor(),
                             target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

classes = {
    0 : 'plane',    1 : 'car',    2 : 'bird',   3 : 'cat',    4 : 'deer',
    5 : 'dog',      6 : 'frog',   7 : 'horse',  8 : 'ship',   9 : 'truck'
}

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# img, label = training_data[1]
train_iter = iter(train_loader)
# img, label = next(train_iter)
# print(img.shape)
# print(label.shape)

# shows image tensor
def show(tens, label):
  # make_grid makes a grid of images from a batch of tensors BxCxHxW
  label = torch.argmax(label,dim=1)
  print(label)
  img = torchvision.utils.make_grid(tens) # flattens B dimension to make a grid
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img, (1,2,0)))     # seems each image is spaced by 2 pixels, channels goes last when plotting
  # title = classes[label]
  title = ' '.join([classes[t.item()] for t in label])
  plt.title(title)

# show(img, label)


In [ ]:
# Make a class for your model

# nn is composed of a bunch of subclasses, myNN inherits from nn.Module
# within a subclass of torch.nn.Module, it's assumed we want to track gradients on the layer's weights
class myNN(nn.Module):

  def __init__(self):
    super().__init__()
    self.flat = nn.Flatten()

    # Sequential class stores modules that will be passed sequentially through constructor
    # input: 64x3x32x32;   output: 64x16x8x8
    self.conv_pool_stack = nn.Sequential(
      nn.Conv2d(3,16,kernel_size=3,padding=1, bias=False),
      nn.ReLU(),
      nn.BatchNorm2d(16),
      nn.Conv2d(16,16,kernel_size=3,padding=1, bias=False),
      nn.ReLU(),
      nn.MaxPool2d(2,stride=2),
      nn.BatchNorm2d(16),
      nn.Conv2d(16,32,kernel_size=3,padding=1,bias=False),
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.Conv2d(32,32,kernel_size=3,padding=1,bias=False),
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.MaxPool2d(2,stride=2),
      nn.BatchNorm2d(32)
    )

    # input: 64x32x8x8;    output: 16x10
    self.linear_relu_stack = nn.Sequential(
      self.flat,           # Flatten class flattens starting at dimension default 1 and ending at dimension default -1 --> 16x2048
      nn.Linear(2048, 512, bias=True),
      nn.ReLU(),
      nn.Linear(512, 64, bias=True),
      nn.ReLU(),
      nn.Linear(64, 10, bias=True),
      nn.Softmax(dim=1)
    )

  def forward(self,x):
    after_conv = self.conv_pool_stack(x)
    logits = self.linear_relu_stack(after_conv)
    return logits

device = "cuda" if torch.cuda.is_available() else "cpu"

# to() creates an instance of the network and moves it to a device
# model = myNN().to(device)
# print(model)



In [ ]:
# print(model.conv_pool_stack[0].weight[0,0])
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

# input = torch.randn((4,3,32,32),requires_grad=False)
# target = torch.randint(0,10,size=(4,))
# logits = model(input)
# loss = loss_fn(logits,target)
# print(loss)
# loss.backward()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.step()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.zero_grad()

In [ ]:
def preprocess(x):
  # find std and mean per channel
  x_stds = torch.sqrt(torch.var(x, dim=(0,2,3)))
  x_means = torch.sum(x, dim=(0,2,3)) / (x.shape[0]*x.shape[2]*x.shape[3])

  for channel in range(3):
    x[:,channel,:,:] -= x_means[channel]
    x[:,channel,:,:] /= x_stds[channel]
  return x

In [ ]:
def accuracy(pred,y):
  guesses = torch.argmax(pred,dim=1)
  matches = sum(guesses==y)
  return matches/torch.numel(y), matches


In [170]:
def train(dataloader, model, loss_fn, optimizer):
  # sets the module to training mode, eval does the opposite
  model.train()

  accs = []
  matches = 0

  for batch, (X, y) in enumerate(dataloader):
    imgs, labels = X.to(device)
    labels = y.to(device)

    # do data preprocessing
    imgs = preprocess(imgs)
    # do not call model.foward(X), doing model(X) calls the __call__() function which does a few extra hooks...
    pred = model(imgs)
    loss = loss_fn(pred, torch.argmax(labels,dim=1))
    acc, m = accuracy(pred,torch.argmax(labels,dim=1))
    matches += m

    if batch % 50 == 0:
      accs.append(acc)
      print(f"Progress: {(batch) * len(X)}, current accuracy: {m/(len(X))}")

    # Backpropagation
    loss.backward()     # backpropagates prediction loss, deposits gradients of the loss w.r.t. for each parameter that has requires_grad=True
    optimizer.step()    # updates the parameter values, "learns"
    optimizer.zero_grad()       # zeros the grads because it is a dynamic graph

  print(f"Epoch Accuracy: {matches/50000}")



In [ ]:
def test(dataloader, model):

  # set module to testing mode, train does the opposite
  model.eval()

  # used to turn off gradient calculations
  with torch.no_grad():
    for batch, (X,y) in enumerate(model):
      imgs = X.to(device)

In [168]:
model = myNN().to(device)

# set cross entropy loss and Adam optimizer functions
loss_fn = nn.CrossEntropyLoss()     # THIS NEEDS TARGET VALUES, NOT ONE-HOT ENCODING
optimizer = torch.optim.Adam(model.parameters())

In [169]:
epochs = 5
start = datetime.now()
for e in range(epochs):
  train(train_loader, model, loss_fn, optimizer)
elapse = datetime.now() - start
print("Training Time: ", elapse)


Progress: 0, current accuracy: 0.078125
Progress: 3200, current accuracy: 0.4375
Progress: 6400, current accuracy: 0.421875
Progress: 9600, current accuracy: 0.375
Progress: 12800, current accuracy: 0.4375
Progress: 16000, current accuracy: 0.375
Progress: 19200, current accuracy: 0.515625
Progress: 22400, current accuracy: 0.515625
Progress: 25600, current accuracy: 0.515625
Progress: 28800, current accuracy: 0.4375
Progress: 32000, current accuracy: 0.46875
Progress: 35200, current accuracy: 0.4375
Progress: 38400, current accuracy: 0.46875
Progress: 41600, current accuracy: 0.453125
Progress: 44800, current accuracy: 0.515625
Progress: 48000, current accuracy: 0.484375
tensor(22289)
16
781
12512
Epoch Accuracy: 1.7814098596572876
